In [ ]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [10]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x,W) + b)
y_ = tf.placeholder(tf.float32, [None,10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(20)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        if i % 100 == 0:
            print ('Test Accuracy:', sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Test Accuracy: 0.1943
Test Accuracy: 0.8674
Test Accuracy: 0.8801
Test Accuracy: 0.8951
Test Accuracy: 0.9009
Test Accuracy: 0.8926
Test Accuracy: 0.9011
Test Accuracy: 0.9047
Test Accuracy: 0.9056
Test Accuracy: 0.9074


In [11]:
# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [20]:
tf.reset_default_graph()
# Create the model
# placeholder
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# first convolutinal layer
w_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second convolutional layer
w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# densely connected layer
w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
w_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

# train and evaluate the model
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
            test_accuracy = sess.run(accuracy, feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0})
            print ("step %d, train accuracy %g, test accuracy %g" %(i, train_accuracy, test_accuracy))
        sess.run(train_step, feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})

step 0, train accuracy 0.14, test accuracy 0.1356
step 100, train accuracy 0.86, test accuracy 0.8431
step 200, train accuracy 0.92, test accuracy 0.9118
step 300, train accuracy 0.84, test accuracy 0.9347
step 400, train accuracy 0.92, test accuracy 0.9417
step 500, train accuracy 0.96, test accuracy 0.9462
step 600, train accuracy 0.9, test accuracy 0.9525
step 700, train accuracy 0.96, test accuracy 0.9565
step 800, train accuracy 0.94, test accuracy 0.9614
step 900, train accuracy 0.92, test accuracy 0.9658
step 1000, train accuracy 0.98, test accuracy 0.9655
step 1100, train accuracy 0.98, test accuracy 0.9662
step 1200, train accuracy 0.96, test accuracy 0.9677
step 1300, train accuracy 0.98, test accuracy 0.9698
step 1400, train accuracy 0.94, test accuracy 0.9732
step 1500, train accuracy 0.96, test accuracy 0.9737
step 1600, train accuracy 0.98, test accuracy 0.973
step 1700, train accuracy 0.98, test accuracy 0.9737
step 1800, train accuracy 1, test accuracy 0.974
step 1900, 

KeyboardInterrupt: 